In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import re
import os
import nltk
import string
import random
import warnings
import numpy                               as np
import pandas                              as pd 
import matplotlib.pyplot                   as plt
import seaborn                             as sns
import plotly.express                      as ex
import plotly.graph_objs                   as go
import plotly.offline                      as pyo
import pymc3                               as pm
import theano.tensor                       as T
from plotly.subplots                       import make_subplots
from sklearn.decomposition                 import TruncatedSVD,PCA

from sklearn.cluster                       import KMeans
from nltk.sentiment.vader                  import SentimentIntensityAnalyzer as SIA
from wordcloud                             import WordCloud,STOPWORDS
from pandas.plotting                       import autocorrelation_plot
from statsmodels.graphics.tsaplots         import plot_acf
from statsmodels.graphics.tsaplots         import plot_pacf
from statsmodels.tsa.seasonal              import seasonal_decompose

from statsmodels.tsa.statespace.sarimax    import SARIMAX
from nltk.util                             import ngrams
from nltk                                  import word_tokenize
from nltk.stem                             import PorterStemmer
from nltk.stem                             import WordNetLemmatizer
from wordcloud                             import STOPWORDS
from tqdm.notebook                         import tqdm

from sklearn.feature_extraction.text       import CountVectorizer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [26]:

%matplotlib inline

pyo.init_notebook_mode()
tqdm.pandas()

warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)


plt.style.use('ggplot')
pyo.init_notebook_mode()
plt.rc('figure',figsize=(18,11))
sns.set_context('paper',font_scale=2)

def set_seed(seed=31415):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    
set_seed()

Output hidden; open in https://colab.research.google.com to view.

In [27]:
%cd /content/drive/MyDrive/7404-Project-Demo/7404Project/Sentiment Analysis
%ls

/content/drive/MyDrive/7404-Project-Demo/7404Project/Sentiment Analysis
 analyst_ratings_processed.csv  'Sentiment Analysis.py'
 Sentiment_Analysis.ipynb


In [28]:
n_data = pd.read_csv('./analyst_ratings_processed.csv',parse_dates=['publish_date'])
n_data.head(3)

,publish_date,headline_text
0,2020-06-11,Johnson & Johnson To Start Coronavirus Vaccine...
1,2020-06-11,80 Biggest Movers From Yesterday
2,2020-06-11,80 Biggest Movers From Yesterday


In [29]:
n_data = n_data.rename(columns={'publish_date':'date','headline_text':'text'})
n_data = n_data.groupby(['date'], as_index = False).agg({'text': ' '.join})
n_data = n_data.drop_duplicates()
n_data.head(3)

,date,text
0,2009-02-14,How Treasuries and ETFs Work
1,2009-04-27,Update on the Luxury Sector: 2nd Quarter 2009
2,2009-04-29,Going Against the Herd


In [30]:
#Sorting And Feature Engineering
n_data['year']         = pd.DatetimeIndex(n_data['date']).year
n_data['month']        = pd.DatetimeIndex(n_data['date']).month
n_data['day']          = pd.DatetimeIndex(n_data['date']).day
n_data['day_of_year']  = pd.DatetimeIndex(n_data['date']).dayofyear
n_data['quarter']      = pd.DatetimeIndex(n_data['date']).quarter
n_data['season']       = n_data.month%12 // 3 + 1

In [31]:
yearly = n_data.groupby(['year'], as_index = False).agg({'text': ' '.join})
monthly = n_data.groupby(['month'], as_index = False).agg({'text': ' '.join})

In [32]:
#Vader Setiment Analysis
sid = SIA()
n_data['sentiments']           = n_data['text'].progress_apply(lambda x: sid.polarity_scores(' '.join(re.findall(r'\w+',x.lower()))))
n_data['Positive Sentiment']   = n_data['sentiments'].progress_apply(lambda x: x['pos']+1*(10**-6)) 
n_data['Neutral Sentiment']    = n_data['sentiments'].progress_apply(lambda x: x['neu']+1*(10**-6))
n_data['Negative Sentiment']   = n_data['sentiments'].progress_apply(lambda x: x['neg']+1*(10**-6))
n_data.drop(columns=['sentiments'],inplace=True)

  0%|          | 0/3944 [00:00<?, ?it/s]

  0%|          | 0/3944 [00:00<?, ?it/s]

  0%|          | 0/3944 [00:00<?, ?it/s]

  0%|          | 0/3944 [00:00<?, ?it/s]

In [ ]:
n_data.head(5)

In [ ]:
n_data.to_csv('./n_data.csv')